In [ ]:
%matplotlib widget

In [ ]:
import matplotlib

import numpy as np
from pathlib import Path
import flammkuchen as fl
import matplotlib.pyplot as plt 

from bouter.utilities import reliability 
from skimage.filters import threshold_otsu
import xarray as xr
from scipy.signal import detrend 

from lavian_et_al_2025.visual_motion.stimulus_functions import stim_vel_dir_dataframe, quantize_directions, get_paint_function
from lavian_et_al_2025.imaging.imaging_classes import LightsheetExperiment

from scipy.signal import argrelextrema
from scipy.signal import find_peaks

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import cm

In [ ]:
master_path = Path(r'\\portulab.synology.me\data\Hagar and Ot\E0040\v10\LS')
fish_list = list(master_path.glob("*f*"))
path = fish_list[7]
print(path)

In [ ]:
suite2p_brain = fl.load(path / "data_from_suite2p_cells_brain.h5")
in_brain_idx = suite2p_brain['coords_idx']
num_traces = np.shape(in_brain_idx)[0]

In [ ]:
traces = fl.load(path / "filtered_traces.h5", "/detr")[:, in_brain_idx]
sens_corr = fl.load(path / "sensory_regressors.h5", "/regressors")

In [ ]:
coords = fl.load(path / "data_from_suite2p_cells.h5")['coords'][in_brain_idx]
anatomy = fl.load(path / "data_from_suite2p_cells.h5")['anatomy_stack']

In [ ]:
reg_values = np.dot(traces.T, sens_corr.values)

In [ ]:
n_t, n_neurons = np.shape(traces)
fs = 2

In [ ]:
mean_traces = np.nanmean(traces.T, axis=0)
mean_regs = np.nanmean(sens_corr.values, axis=0)

traces_std = np.nanstd(traces.T, axis=0)
regs_std = np.nanstd(sens_corr.values, axis=0)


# Normalize the data
traces_normalized = (traces.T - mean_traces) / traces_std
regs_normalized = (sens_corr.values - mean_regs) / regs_std
    
dot_product = np.dot(traces_normalized, regs_normalized)
reg_corr_values = dot_product / (n_t - 1)

In [ ]:
exp = LightsheetExperiment(path)
stimulus_log = exp.stimulus_log
stim_value, t_values = get_paint_function(stimulus_log, 'E0040_motions_cardinal')
stim_value = stim_value / 255
num_stim = np.shape(stim_value)[0]

In [ ]:
fig, ax = plt.subplots(2 ,1, figsize=(10, 5), gridspec_kw={'height_ratios': [5,1]})
plt.subplots_adjust(left=0.05)
for i in range(num_stim):
    ax[0].axvspan(
        t_values[i, 0] - t_values[0,0] + 10,
        t_values[i, 1] - t_values[0,0] + 10,
        facecolor=[stim_value[i, 0], stim_value[i, 1], stim_value[i, 2]],
        alpha=0.5,
    )

In [ ]:
t_imaging = np.arange(n_t) / fs

high_corr_traces = np.zeros(8)
for direction in range(8):
    ind_trace = np.nanargmax(reg_corr_values[:,direction])
        
    high_corr_traces[direction] = int(ind_trace)
    
    ax[0].plot(t_imaging, traces[:, int(ind_trace)] + (10 * direction), color='k')

In [ ]:
beh_log = exp.behavior_log
tail = beh_log['tail_sum']
t_beh = beh_log['t']

### cleaning tail
tail_noise = np.where(np.abs(tail) < 0.2)[0]
clean_tail = np.copy(tail)
clean_tail[tail_noise] = 0

ax[1].plot(t_beh, clean_tail - 10, color='k')
ax[1].axis('off')
ax[0].axis('off')

In [ ]:
file_name = "most corr traces.jpg"
fig.savefig(path / file_name, dpi=300)
file_name = "most corr traces.pdf"
fig.savefig(path / file_name, dpi=300)

In [ ]:
high_corr_traces = high_corr_traces.astype(int)

In [ ]:
#Make color arr for example ROIs
col_order = [6, 1, 3, 7, 0, 5, 4, 2]
roi_cols = np.array([[stim_value[i, 0], stim_value[i, 1], stim_value[i, 2]] for i in col_order])

In [ ]:
morphed_coords = fl.load(path / 'registration' / 'to_h2b_baier_ref' / 'antspy' / 'mov_coords_transformed.h5')
ref_anatomy = fl.load(path / 'registration' / 'to_h2b_baier_ref' / 'antspy' / 'ref_mapped.h5')
mov_mapped = fl.load(path / 'registration' / 'to_h2b_baier_ref' / 'antspy' / 'mov_mapped.h5')
mov_coords_mapped = fl.load(path / 'registration' / 'to_h2b_baier_ref' / 'antspy' / 'mov_roi_coords_mapped.h5')


morphed_coords = morphed_coords[in_brain_idx]
mov_coords_mapped = mov_coords_mapped[in_brain_idx]

In [ ]:
########### make a nice anatomy figure showing the cells location
### AVERAGE REFERENCE ANATOMY
fig_anat, ax_anat = plt.subplots(1,2, figsize=(5,5), sharey=True)
ax_anat[0].imshow(np.nanmean(ref_anatomy, 2).T, cmap="gray_r")
ax_anat[1].imshow(np.nanmean(ref_anatomy, 0), cmap="gray_r")

for i in range(8):
    ax_anat[0].scatter(morphed_coords[high_corr_traces[i],0], morphed_coords[high_corr_traces[i],1], color=roi_cols[int(i)], s=10)
    ax_anat[1].scatter(morphed_coords[high_corr_traces[i],2], morphed_coords[high_corr_traces[i],1], color=roi_cols[int(i)], s=10)


In [ ]:
########### make a nice anatomy figure showing the cells location
### AVERAGE INTERPOLATED ANATOMY
fig_anat, ax_anat = plt.subplots(1,2, figsize=(5,5), sharey=True)
ax_anat[0].imshow(np.nanmean(mov_mapped, 2).T, cmap="gray_r")
ax_anat[1].imshow(np.nanmean(mov_mapped, 0), cmap="gray_r")

for i in range(8):
    ax_anat[0].scatter(mov_coords_mapped[high_corr_traces[i],0], mov_coords_mapped[high_corr_traces[i],1], color=roi_cols[int(i)], s=10)
    ax_anat[1].scatter(mov_coords_mapped[high_corr_traces[i],2], mov_coords_mapped[high_corr_traces[i],1], color=roi_cols[int(i)], s=10)


In [ ]:
########### make a nice anatomy figure showing the cells location
### AVERAGE SLICES
zmin = coords[high_corr_traces][:, 0].min()
zmax = coords[high_corr_traces][:, 0].max()
zmin, zmax

steps = 3
z_step = (zmax-zmin)/steps
z_step

fig_anat, axes = plt.subplots(1,steps, figsize=(9, 6))

for i, plane in enumerate(np.arange(zmin, zmax, z_step)):
    axes[int(i)].imshow(anatomy[int(plane):int(plane+z_step)].mean(0), cmap="gray_r", aspect='equal')
    
    sel_rois = np.nonzero(np.logical_and(coords[high_corr_traces][:, 0]>=plane, coords[high_corr_traces][:, 0]<(plane+z_step)))
    axes[int(i)].scatter(coords[high_corr_traces[sel_rois[0]],2], coords[high_corr_traces[sel_rois[0]],1], edgecolor=roi_cols[sel_rois[0]], facecolor='none', linewidth=1.5, s=20)
    
for ax in axes:
    ax.invert_yaxis()
    ax.axis('off')
    
scale_bar_len = 100
scale_bar_xpos = 25
scale_bar_ypos1 = 50
fs = 8
microns_per_px = 0.6

axes[0].plot((scale_bar_xpos, scale_bar_xpos+(scale_bar_len/microns_per_px)), (scale_bar_ypos1, scale_bar_ypos1), c='black')
plt.tight_layout()

In [ ]:
fig_anat.savefig(path / 'Location of example neurons_v2.png', dpi=300)

In [ ]:
piezo_min = exp['imaging']['microscope_config']['lightsheet']['scanning']['z']['piezo_min']
piezo_max = exp['imaging']['microscope_config']['lightsheet']['scanning']['z']['piezo_max']
n_planes = exp['imaging']['microscope_config']['lightsheet']['scanning']['triggering']['n_planes']

microns_per_plane = (piezo_max - piezo_min)/n_planes

In [ ]:
z_step*microns_per_plane

In [ ]:
color_list = ['k', 'k', 'k', 'k', 'k', 'k', 'k', 'k']
title_list = ['→', '↘', '↓', '↙', '←', '↖', '↑', '↗']

In [ ]:
high_corr_traces = high_corr_traces.astype(int)

In [ ]:
############## plot tuning curves for selected neurons 

In [ ]:
# load reordered traces 
trial_traces = fl.load(path / 'reliability_index_arr.h5')['trial_traces']

In [ ]:
selected_traces = trial_traces[high_corr_traces]
selected_traces_avg = np.nanmean(selected_traces, axis=1)

In [ ]:
tuning_curve_selected = np.zeros((8,8)) #n_neurons, n_dirs
trial_len = 20

for i in range(8):
    t1 = i*trial_len*2 + trial_len
    t2 = t1 + trial_len    
    tuning_curve_selected[:,i] = np.nanmean(selected_traces_avg[:, t1:t2], axis=1)

In [ ]:
fig3, ax3 = plt.subplots(8 ,1, figsize=(2.5, 10))
fig3.subplots_adjust(hspace=0.2, bottom=0.1, top=0.9, left=0.2)

t_imaging = np.arange(n_t) / fs

high_corr_traces = np.zeros(8)
for direction in range(8):
    ind_trace = np.nanargmax(reg_corr_values[:,direction])
        
    high_corr_traces[direction] = int(ind_trace)
    
    ax3[7-direction].plot(tuning_curve_selected[direction], color='k')
    ax3[7-direction].scatter(np.arange(8), tuning_curve_selected[direction], color='k')
    ax3[7-direction].axis('off')


In [ ]:
file_name = "example tuning curve.jpg"
fig3.savefig(path / file_name, dpi=300)
file_name = "example tuning curve.pdf"
fig3.savefig(path / file_name, dpi=300)